In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import rhinoMorph
from konlpy.tag import Okt
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
peter_df = pd.read_csv('./articles_final_peter.csv')
peter_df

,media,date,title,article_original,url
0,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,경향신문,2020-06-08,BTS 유튜브 주최 가상 졸업식 디어 클래스 오브 2020 축사 꽃다발은 없어도 가...,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,경향신문,2020-07-23,BTS 75만명 본 방방콘 더 라이브로 기네스 세계기록 추가,방탄소년단의 온라인 콘서트 방방콘 더 라이브이하 방방콘가 세계에서 가장 많은 시청...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-08-30,방탄소년단 신곡 영국 오피셜 차트 3위,그룹 방탄소년단BTS사진의 신곡 다이너마이트Dynamite가 세계 양대 팝 차트 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,경향신문,2020-09-29,3주 만에 다시 빌보드 정상에 선 방탄소년단,그룹 방탄소년단BTS사진의 다이너마이트가 3주 만에 미국 빌보드 메인 싱글 차트인...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-10-13,병무청 대중문화우수자 병역 연기 입법 추진확정시 BTS 연기 가능,병무청이 13일 대중문화예술 분야 우수자의 병역 징집소집 연기 등을 담은 병역법 ...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [38]:
peter_df.article_original[10]

' 그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 올랐다. 한국 대중음악가수 최초의 기록이다. 전문가들은 방탄소년단이 K팝 아티스트를 넘어 미국 주류 음악계의 일원으로 자리매김했다고 평가했다. 그래미 어워즈를 주관하는 미국 레코딩 아카데미는 24일현지시간 방탄소년단의 다이너마이트를 제63회 그래미 어워즈 베스트 팝 듀오그룹 퍼포먼스 부문 후보로 발표했다. 국내 클래식이나 국악 관계자가 후보에 오르거나 수상한 적은 있었지만 한국 대중음악 가수가 후보로 지명된 것은 방탄소년단이 처음이다. 베스트 팝 듀오그룹 퍼포먼스는 팝 장르 세부 시상 분야 중 하나로 2012년 신설됐다. 듀오그룹컬래버레이션 형태로 팝 보컬이나 연주 퍼포먼스에서 뛰어난 예술적 성취를 거둔 뮤지션을 선정한다. 방탄소년단의 다이너마이트는 제이 발빈두아 리파배드 버니테이니의 언 디아 저스틴 비버와 퀘이보의 인텐션스 레이디 가가와 아리아나 그란데의 레인 온 미 테일러 스위프트와 본 이베어의 엑사일 등 쟁쟁한 후보들과 경쟁하게 된다. 방탄소년단은 25일 공식 트위터를 통해 힘든 시기 우리의 음악을 들어주시고 공감해주신 모든 분들께 감사하다며 무엇보다 그래미 후보 아티스트라는 기적을 만들어주신 건 아미 여러분이다라는 소감을 밝혔다. 후보 발표 직후 환호성을 지르는 뷔 정국 RM 지민의 모습이 담긴 동영상도 공개했다. 1974년 시작된 그래미 어워즈는 미국 음악상 중에서도 최고 영예의 상으로 꼽힌다. 가수 프로듀서 엔지니어 평론가 등 음악계 전문가들로 구성된 미국레코드예술과학아카데미NARAS 회원들이 투표를 통해 후보 및 수상자를 선정한다. 이 때문에 후보 지명만으로도 업계 관계자들의 인정을 받았다는 의미를 지닌다. 정민재 대중음악평론가는 미국인들에게 방탄소년단은 한국에서 온 슈퍼스타라는 이미지가 강했다며 이번 후보 지명은 방탄소년단이 미국 음악시장의 한 일원으로 자리매김했다는 의미가 있다고 말했다. 김도헌 대중음악웹진 이즘 편집장도 방탄소년단은 전체 대중이 아니라 자신들의 팬덤을

In [3]:
jm_df = pd.read_csv('./extData_LDA_ver2.csv')
jm_df

,media,date,title,article_original,url
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,중앙일보,2020-05-25,한식구 된 BTS세븐틴여자친구엔터업계 지각변동 올까,방탄소년단BTS 소속사 빅히트엔터테인먼트가 뉴이스트세븐틴 등이 소속된 플레디스 엔...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-06-08,방탄소년단의 온라인 축사 우리도 서툰 20대우리가 위로와 희망이 되고 싶다,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,경향신문,2020-06-15,BTS 온라인 콘서트 방방콘 The Live로 107개 지역 75만 아미 만났다,그룹 방탄소년단이 실시간 라이브 공연 방방콘 The Live를 열어 75만 660...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,경향신문,2020-09-10,BTS가 전 세계에 전하는 서울의 매력,서울시가 방탄소년단BTS이 서울을 7가지 매력으로 소개하는 영상을 11일 공개한다...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-09-15,방탄소년단 미국 빌보드 싱글 차트 2위3주째 최상위권,그룹 방탄소년단BTS의 다이너마이트Dynamite가 미국 빌보드 메인 싱글 차트에...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달 56일 코스피 일반 청약 신주 713만주 중 142만주 배정 청약증거금 58...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,중앙일보,2020-09-20,BTS 행사 연출했다는 탁현민허은아 나르시시즘 정신차려라,허은아 국민의힘 의원이 문재인 대통령과 방탄소년단BTS이 만남을 연출한 탁현민 청...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [4]:
will_df = pd.read_csv('./Willver2_rhino10.csv')
will_df

,media,date,title,article_original,url
0,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,한겨레,2020-06-04,BTS 인종차별과 폭력에 반대우리 모두 존중받을 권리 있어,그룹 방탄소년단BTS이 인종차별과 폭력에 반대한다는 뜻을 밝혔다. 미국에서 백인 경...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-09-01,방탄소년단 한국 가수 최초 빌보드 싱글 1위K팝 새역사,2012년 싸이 강남스타일 2위 기록 넘어서방탄소년단BTS이 한국 가수 중 처음으로...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-09-03,빅히트 공모주 청약 앞두고 BTS에 주식 증여상장 시 1인당 최대 200억원대,경향신문방탄소년단BTS이 소속사인 빅히트엔터테인먼트로부터 1인당 7만주 가까운 보통...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,조선일보,2020-09-20,다이너마이트 영어로 부른 BTS 방탄 노래는 들려 따라갈 수 있겠다 영상,동영상 뉴스문재인 대통령이 방탄소년단BTS의 노래에 대해 그들의 노래와 춤 모두를 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인은 아...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,중앙일보,2020-10-14,14억 시장에 휘둘린 BTS로이터 중국은 정치적 지뢰,중국 관영 매체 환구시보가 방탄소년단BTS의 수상 소감이 중국 네티즌의 분노를 일으...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,중앙일보,2020-10-20,택배사 또 BTS제품 거부우리가 했겠나 이유 알지 않나,중국 내 5위 규모의 물류업체 윈다가 방탄소년단BTS 관련 제품 배송을 중단하겠다고...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,경향신문방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [5]:
articles_df = pd.concat([peter_df,jm_df,will_df])
articles_df.sort_values(by='date',inplace=True)

In [6]:
articles_df.reset_index(inplace=True)
articles_df.drop('index',inplace=True,axis=1)
articles_df

,media,date,title,article_original,url
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...
3,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...
7,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...
8,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...


In [ ]:
#불용어 import
stopwords2 = pd.read_csv('./불용어리스트2.txt',header=None)
stopwords = list(stopwords2[0])
stopwords

In [8]:
#형태소 토크나이징
def okt_morph(articles):
    okt = Okt()
    articles_morphed = []
    for article in articles:
    #     morph = okt.morphs(article)
        morph = okt.nouns(article)

        stopped = ''
        for word in morph:
            if word not in stopwords:
                stopped += word
                stopped += ' '
                
        articles_morphed.append(stopped)
    return articles_morphed

In [9]:
#tokenizing
okt = Okt()

df = okt_morph(articles_df.article_original)
articles_df['morphed'] = df
articles_df

,media,date,title,article_original,url,morphed
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 서울 시작 세계 투어 돌입 방탄소년단 오전 빅히트 공식 채널 통해 맵 오...
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 한국 가수 최초 미국 그래미 싱 공연 방탄소년단 날 미국 로스앤젤레...
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...,커넥트 서울 철학 현대 미술 작품 세계 프로젝트 서울 개막 강 이연 참여 인종 세대...
3,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 발매 세계 개 지역 아이튠즈 차트 위 방탄소년단 앨범 중 ...
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,일부 팬 환불 성금 기탁 트와이스 공연 취소 코로나 확산 예정 그룹 방탄소년단 월드...
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 맵 오브 더 솔 영국 오피 셜 차트 빌보드 팝 차트 위 한...
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 예...
7,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 코로나 확산 북미 투어 일정 연기 소속사 빅히트 엔터테인먼트 달 예...
8,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 사진 앨범 누적 판매량 한국 가요 역사상 가장 음반 판 가수 이름 ...
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,최근 소문 이태원 발 코로나 집단 감염 기간 이태원 방문 아이돌 그룹 방탄소년단 정...


In [10]:
tfidf = TfidfVectorizer(okt.nouns,lowercase=False)
article_tfidf = tfidf.fit_transform(articles_df.article_original)
article_tfidf

C:\Users\user\anaconda3\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass input=<bound method Okt.nouns of <konlpy.tag._okt.Okt object at 0x000002D12019CD60>> as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


<39x5560 sparse matrix of type '<class 'numpy.float64'>'
	with 9170 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.cluster import KMeans

n_clusters = 10
km_cluster = KMeans(n_clusters=n_clusters, random_state=42)
km_cluster.fit(article_tfidf)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

In [12]:
articles_df['cluster_label'] = cluster_label
articles_df

,media,date,title,article_original,url,morphed,cluster_label
0,조선일보,2020-01-22,방탄소년단 4월 월드투어 돌입 서울부터 스페인까지,방탄소년단BTS이 오는 4월 서울을 시작으로 세계 투어에 돌입한다. 방탄소년단은 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 서울 시작 세계 투어 돌입 방탄소년단 오전 빅히트 공식 채널 통해 맵 오...,4
1,조선일보,2020-01-27,방탄소년단 한국 가수 최초 그래미 공연 내년엔 후보 도전,그룹 방탄소년단BTS이 26일현지 시각 한국 가수 최초로 미국 그래미 시싱식에서 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 한국 가수 최초 미국 그래미 싱 공연 방탄소년단 날 미국 로스앤젤레...,8
2,조선일보,2020-01-29,7명의 몸짓을 예술로 현대미술이 BTS라는 물길을 만났다,커넥트 BTS서울 BTS 철학 담은 현대미술 작품 세계 5개도시서 펼치는 프로젝트2...,https://news.naver.com/main/read.nhn?mode=LSD&...,커넥트 서울 철학 현대 미술 작품 세계 프로젝트 서울 개막 강 이연 참여 인종 세대...,5
3,조선일보,2020-02-22,방탄소년단 새 앨범 91개국 아이튠즈 1위 역대 최고기록,그룹 방탄소년단BTS 새 앨범이 발매하자마자 세계 91개 지역 아이튠즈 차트 1위...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 발매 세계 개 지역 아이튠즈 차트 위 방탄소년단 앨범 중 ...,1
4,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,일부 팬 환불 성금 기탁 트와이스 공연 취소 코로나 확산 예정 그룹 방탄소년단 월드...,4
5,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 맵 오브 더 솔 영국 오피 셜 차트 빌보드 팝 차트 위 한...,1
6,조선일보,2020-03-16,방탄소년단 빌보드 200 8위3주 연속 10위권,그룹 방탄소년단BTS이 미국 빌보드 메인 앨범 차트인 빌보드 200에서 8위를 차...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 빌보드 메인 앨범 차트 빌보드 위 차지 주 연속 위 빌보드 예...,1
7,경향신문,2020-03-27,방탄소년단 코로나19 확산에 북미 투어 연기,그룹 방탄소년단BTS이 코로나19 확산으로 북미 투어 일정을 연기한다. 소속사 빅...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 코로나 확산 북미 투어 일정 연기 소속사 빅히트 엔터테인먼트 달 예...,4
8,경향신문,2020-04-09,방탄소년단 음반 누적판매 2032만장한국 가요 최다 기록,그룹 방탄소년단사진이 앨범 누적 판매량 2000만장을 넘기면서 한국 가요 역사상 ...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 사진 앨범 누적 판매량 한국 가요 역사상 가장 음반 판 가수 이름 ...,1
9,조선일보,2020-05-18,이태원 아이돌 BTS 정국 아스트로 차은우 NCT 재현 세븐틴 민규였다,최근 소문으로만 떠돌던 이태원발 코로나 집단 감염 기간 이태원 방문 아이돌이 그룹...,https://news.naver.com/main/read.nhn?mode=LSD&...,최근 소문 이태원 발 코로나 집단 감염 기간 이태원 방문 아이돌 그룹 방탄소년단 정...,5


In [13]:
top_list = []
for i in range(10):
    d = km_cluster.transform(article_tfidf)[:,i]
    ind = np.argsort(d)[::][:1]
    top_list.append(ind[0])
top_list

[28, 5, 36, 13, 4, 26, 21, 25, 34, 24]

In [14]:
final_df = articles_df.iloc[top_list].sort_values(by='date')

final_df.reset_index(inplace=True)
final_df.drop('index',inplace=True,axis=1)
final_df

,media,date,title,article_original,url,morphed,cluster_label
0,경향신문,2020-02-28,BTS 월드투어 4월 서울 콘서트 취소 코로나19 확산 비상,일부 팬들 환불받아 성금 기탁 트와이스 등 공연 취소 잇따라 코로나19의 확산으...,https://news.naver.com/main/read.nhn?mode=LSD&...,일부 팬 환불 성금 기탁 트와이스 공연 취소 코로나 확산 예정 그룹 방탄소년단 월드...,4
1,한겨레,2020-03-02,BTS 4연속 빌보드 정상비틀스 이래 최단기간 달성 그룹,그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 새 앨범 맵 오브 더 솔 영국 오피 셜 차트 빌보드 팝 차트 위 한...,1
2,경향신문,2020-06-08,방탄소년단의 온라인 축사 우리도 서툰 20대우리가 위로와 희망이 되고 싶다,그룹 방탄소년단BTS이 코로나19 확산으로 오프라인 졸업식에 가지 못하는 졸업생들...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 코로나 확산 오프라인 졸업식 가지 졸업생 위해 축사 전 리더 꽃다발...,3
3,조선일보,2020-09-17,방탄소년단 빅히트 공모주 다이너마이트 터지나,내달 56일 코스피 일반 청약 신주 713만주 중 142만주 배정 청약증거금 58...,https://news.naver.com/main/read.nhn?mode=LSD&...,내달 코스피 일반 청약 신주 중 배정 청약 증거 금 코스닥 시장 등판 카카오 게임 ...,6
4,경향신문,2020-09-24,BTS 필사적으로 자신을 사랑하고 미래를 상상하려 노력하세요,2년 만에 유엔 화상 총회 참여 코로나 위기 속 연대의 목소리 지금이야말로 우리...,https://news.naver.com/main/read.nhn?mode=LSD&...,유엔 화상 총회 참여 코로나 위기 속 연대 목소리 지금 스스로 얼굴 마주 필사 사랑...,9
5,경향신문,2020-09-29,3주 만에 다시 빌보드 정상에 선 방탄소년단,그룹 방탄소년단BTS사진의 다이너마이트가 3주 만에 미국 빌보드 메인 싱글 차트인...,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 사진 다이너마이트 주 미국 빌보드 메인 싱글 차트 핫 위 다시 차지...,7
6,중앙일보,2020-10-07,아미도 냅두라는데BTS 병역특례 2년만에 다시 띄우는,BTS를 병역특례 해주자는 입장이 아니다. 순수예술인은 특례대상인데 대중예술인...,https://news.naver.com/main/read.nhn?mode=LSD&...,병역 특례 입장 예술인 특례대상 대중 예술인 대한 형평성 문제 사례 제기 안민석 문...,5
7,중앙일보,2020-10-13,중국엔 정치적 지뢰 있다BTS 지우기에 깜짝 놀란 외신,방탄소년단BTS의 밴플리트상 수상 소감을 두고 중국발 논란이 커지면서 주요 외신들...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 리트 수상 소감 중국 발 논란 주요 외신 주목 일부 중국 네티즌 국가 존...,0
8,경향신문,2020-11-25,BTS K팝 최초 그래미 후보 지명가는 길마다 새 역사,그룹 방탄소년단BTS이 미국 최고 권위 음악시상식인 그래미 어워즈 후보에 올랐다....,https://news.naver.com/main/read.nhn?mode=LSD&...,그룹 방탄소년단 미국 최고 권위 음악 시상식 그래미 어워즈 후보 한국 대중음악 가수...,8
9,경향신문,2020-12-01,한글 노래가 빌보드 싱글차트 1위에 방탄소년단 세계 팝시장의 상수가 되다,방탄소년단BTS이 한국어 곡으로 미국 빌보드 싱글 차트 정상에 올랐다. 빌보드 6...,https://news.naver.com/main/read.nhn?mode=LSD&...,방탄소년단 한국어 곡 미국 빌보드 싱글 차트 정상 빌보드 역사상 최초 기록 빌보드 ...,2


In [16]:
final_df.article_original[1]

'그룹 방탄소년단BTS의 새 앨범 맵 오브 더 솔7MAP OF THE SOUL7이 영국 오피셜 차트에 이어 빌보드까지 양대 팝 차트 1위에 올랐다. 한국 가수로서는 최초로 오피셜 차트에서 두 번 연속 정상에 오른 것은 물론 빌보드에서는 비틀스 이후 최단기간 동안 4개 앨범을 1위에 올려놓은 가수라는 기록을 쓰게 됐다. 미국 빌보드는 1일현지시각 방탄소년단의 새 앨범 맵 오브 더 솔7이 메인 앨범 차트인 빌보드 200 최신 차트3월7일자에서 1위에 올랐다고 밝혔다. 이번 앨범은 지난달 28일 영국독일프랑스 등의 오피셜 앨범차트 1위에 올랐고 일본 오리콘차트 1위도 차지했다. 유럽과 일본을 넘어 미국까지 글로벌 앨범 시장을 모두 석권한 셈이다. 빌보드 200은 빌보드 양대 차트빌보드 200 핫 100 중 하나로 메인 앨범 순위를 매기는 차트다. 실물 음반 판매량에다 스트리밍 횟수와 디지털 음원 다운로드 횟수를 각각 앨범 판매량으로 환산한 수치를 합산해 순위를 매긴다. 음원 10곡을 다운받거나 1500곡을 스트리밍한 경우 음반 1장을 산 것으로 간주한다. 빌보드가 인용한 닐슨뮤직 데이터의 자료를 보면 지난달 21일 발매된 이 앨범은 27일까지 총 42만2000장의 판매고를 올렸다. 이 가운데 실물 앨범 판매량은 34만7000장이다. 이외에 스트리밍 횟수를 앨범 판매량으로 환산한 수치가 4만8000장 디지털 음원 다운로드 횟수를 앨범 판매량으로 환산한 수치는 2만6000장으로 집계됐다. 방탄소년단은 이번 1위로 아시아에선 처음으로 네 장의 앨범을 연이어 빌보드 정상에 올린 가수가 됐다. 방탄소년단은 2018년 6월 러브 유어셀프 전 티어로 빌보드 200 1위에 처음 올랐다. 이후 2018년 9월 러브 유어셀프 결 앤서 지난해 4월 맵 오브 더 솔 페르소나에 이어 이번 앨범까지 1년9개월 만에 4회 연속 빌보드 200 1위를 차지했다. 빌보드는 방탄소년단이 영국의 전설적인 밴드 비틀스 이래 최단기간 동안 빌보드 200에서 4개 앨범 1위를 달성한 가수가 됐다고 전했다.

Summarization

In [15]:
from transformers import *
from summarizer import Summarizer
from gensim.summarization.summarizer import summarize
import kss
from lexrankr import LexRank

ModuleNotFoundError: No module named 'transformers'

In [61]:
from gensim.summarization import summarize

for article in final_df.article_original:
    print(summarize(article,0.2))
    print('------------------------------------------')


일부 팬들 환불받아 성금 기탁  트와이스 등 공연 취소 잇따라 코로나19의 확산으로 오는 4월 예정됐던 그룹 방탄소년단BTS의 월드투어 서울 콘서트가 취소됐다.
빅히트 측은 코로나19의 전 세계적인 확산으로 현재 4월 공연 시점의 감염병 확산 상황이 예측 불가능하고 공연 인력과 장비 등 국가 간 이동의 불확실성도 높아지고 있다면서 20만 관람객과 아티스트의 건강과 안전을 고려하고 만에 하나 공연일에 임박해 취소해야 할 경우 해외 관람객 및 공연 관련 업체 스태프들의 피해가 커질 수 있어 불가피하게 공연 일정을 1개월여 앞둔 지금 시점에 취소를 결정했다고 설명했다.
------------------------------------------
한국 가수로서는 최초로 오피셜 차트에서 두 번 연속 정상에 오른 것은 물론 빌보드에서는 비틀스 이후 최단기간 동안 4개 앨범을 1위에 올려놓은 가수라는 기록을 쓰게 됐다.
이번 앨범은 지난달 28일 영국독일프랑스 등의 오피셜 앨범차트 1위에 올랐고 일본 오리콘차트 1위도 차지했다.
실물 음반 판매량에다 스트리밍 횟수와 디지털 음원 다운로드 횟수를 각각 앨범 판매량으로 환산한 수치를 합산해 순위를 매긴다.
------------------------------------------
BTS는 한국시간으로 8일 오전 4시 유튜브로 중계된 가상 졸업식 디어 클래스 오브 2020Dear Class of 2020에 참석했다.
디어 클래스 오브 2020은 코로나19 여파로 오프라인 졸업식에 가지 못하는 졸업생들을 위해 유튜브가 주최한 온라인 졸업식이다.
RM은 대한민국 서울에서 온 여러분과 같은 20대 청춘으로서 축하의 말씀을 전한다며저와 저희 멤버들이 하는 얘기가 어떤 식으로든 위로와 희망이 되고 영감이 되면 좋겠다고 말했다.
또 중학교 시절 눈 내리는 졸업식 때 찍은 사진을 학창 시절 내내 메신저 메인사진으로 걸어뒀다며 그때의 두려움과 벅찬 마음이야말로 나 자신의 진짜 모습이었기 때문에 항상 그런 마음으로 살고 싶었기 때문이 아닐까 

In [62]:
from lexrankr import LexRank
import warnings
warnings.filterwarnings('ignore')

for article in enumerate(final_df.article_original):
    lexrank = LexRank()
    lexrank.summarize(article)
    summaries = lexrank.probe(3)
    print()
    print(summaries)
    print('------------------------------------')

AttributeError: 'tuple' object has no attribute 'strip'